In [1]:
!uv pip install dist/inmemory-0.1.0-py3-none-any.whl

Audited 1 package in 10ms


ImportError: cannot import name 'inmry' from 'inmemory' (/Users/shrijayan.rajendran/projects/personal/dash-project-mem/inmemory/inmemory/inmemory/__init__.py)

In [3]:
# Cell 1: Installation and Import Test
print("=== InMemory Self-Hosted Service Test ===")
print("Testing local installation...")

try:
    from inmemory import Memory

    print("✅ Import successful: Memory class loaded")
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("Solution: pip install -e /path/to/inmemory/")
    exit()

# Cell 2: Initialize Memory
print("\n1. Initializing Memory (Self-Hosted)...")
try:
    memory = Memory()
    print("✅ Memory initialized successfully")
    print(f"Memory instance: {type(memory)}")
except Exception as e:
    print(f"❌ Memory initialization failed: {e}")
    exit()

# Cell 3: Add Memories
print("\n2. Adding memories...")
memories_to_add = [
    {
        "memory_content": "Had lunch with Sarah at the Italian restaurant downtown",
        "tags": "food,social,restaurant",
        "people_mentioned": "Sarah",
        "topic_category": "personal",
    },
    {
        "memory_content": "Completed quarterly report for Q3 2024 ahead of deadline",
        "tags": "work,report,achievement",
        "topic_category": "business",
    },
    {
        "memory_content": "Learned about vector databases and embedding techniques",
        "tags": "learning,technology,AI",
        "topic_category": "education",
    },
]

added_memories = []
for i, mem_data in enumerate(memories_to_add, 1):
    try:
        result = memory.add(**mem_data)
        print(f"✅ Memory {i} added: {result.get('memory_id', 'Success')}")
        added_memories.append(result)
    except Exception as e:
        print(f"❌ Failed to add memory {i}: {e}")

# Cell 4: Search Memories
print("\n3. Searching memories...")
search_queries = ["lunch", "report", "learning"]

for query in search_queries:
    try:
        results = memory.search(query)
        print(f"✅ Search '{query}': {len(results.get('results', []))} results")
        for result in results.get("results", [])[:1]:  # Show first result
            content = (
                result.get("content", "N/A")[:50] + "..."
                if len(result.get("content", "")) > 50
                else result.get("content", "N/A")
            )
            print(f"   - {content}")
    except Exception as e:
        print(f"❌ Search '{query}' failed: {e}")

# Cell 5: Advanced Features
print("\n4. Testing advanced features...")

# Tag search
try:
    tag_results = memory.search_by_tags("work")
    print(f"✅ Tag search 'work': {len(tag_results.get('results', []))} results")
except Exception as e:
    print(f"❌ Tag search failed: {e}")

# Get all memories
try:
    all_memories = memory.get_all()
    print(f"✅ Get all memories: {len(all_memories.get('results', []))} total")
except Exception as e:
    print(f"❌ Get all failed: {e}")

# Health check
try:
    health = memory.health_check()
    print(f"✅ Health check: {health.get('status', 'OK')}")
except Exception as e:
    print(f"❌ Health check failed: {e}")

print("\n🎉 Self-hosted service test completed!")

=== InMemory Self-Hosted Service Test ===
Testing local installation...
✅ Import successful: Memory class loaded

1. Initializing Memory (Self-Hosted)...
✅ Memory initialized successfully
Memory instance: <class 'inmemory.memory.Memory'>

2. Adding memories...
✅ Memory 1 added: e88dadc5-f32b-4b91-a0ed-c00bbc37232d
✅ Memory 2 added: 4d544b22-32b7-46b6-97b5-7ecd152ee7d9
✅ Memory 3 added: fbfeefdc-cf97-4a7f-bc0c-6893206ec585

3. Searching memories...
✅ Search 'lunch': 2 results
   - N/A
✅ Search 'report': 2 results
   - N/A
✅ Search 'learning': 2 results
   - N/A

4. Testing advanced features...
✅ Tag search 'work': 0 results
✅ Get all memories: 6 total
✅ Health check: healthy

🎉 Self-hosted service test completed!


In [2]:
!export INMEM_API_KEY="im_c305c0a3aed70265bd4c881c1e361caa"

In [3]:
#!/usr/bin/env python3
"""
Test script to verify the API key validation fix for managed service.

This script tests that the Inmry client can now properly validate API keys
using the correct /v1/users/current endpoint instead of the broken /v1/ping/ endpoint.
"""

import sys

from inmemory import Inmry


def test_managed_service_fix():
    """Test that the managed service API key validation works correctly."""

    print("=== Testing Fixed Managed Service ===")
    print("Testing API key validation fix...")

    # Test API key (use your generated key)
    api_key = "im_c305c0a3aed70265bd4c881c1e361caa"

    try:
        print("\n1. Testing Inmry initialization with API key...")
        print(f"   API Key: {api_key[:15]}...")
        print("   Host: http://localhost:8081")

        # This should now work with the fix!
        inmry = Inmry(api_key=api_key, host="http://localhost:8081")

        print("✅ Inmry client initialized successfully!")
        print(f"   User ID: {inmry.user_id}")
        print(f"   User Info: {inmry.user_info}")

        # Test a basic operation
        print("\n2. Testing health check...")
        health = inmry.health_check()
        print(f"✅ Health check successful: {health}")

        # Test adding a memory
        print("\n3. Testing memory addition...")
        result = inmry.add(
            memory_content="I am happy it is working",
            user_id=inmry.user_info.get("user_id", "test_user"),
            tags="test,bug-fix,validation",
        )
        print(f"✅ Memory added: {result}")

        # Test searching
        print("\n4. Testing memory search...")
        search_results = inmry.search(
            query="validation fix", user_id=inmry.user_info.get("user_id", "test_user")
        )
        print(f"✅ Search completed: {len(search_results.get('results', []))} results")

        # Clean up
        inmry.close()

        print(
            "\n🎉 All tests passed! The API key validation fix is working correctly."
        )
        return True

    except ValueError as e:
        if "API key validation failed" in str(e):
            print(f"❌ API key validation still failing: {e}")
            print("\nPossible issues:")
            print("1. API server not running on localhost:8000")
            print("2. API key is invalid or expired")
            print("3. /v1/users/current endpoint not working")
            return False
        print(f"❌ Unexpected error: {e}")
        return False

    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return False


def test_comparison():
    """Show the before/after comparison."""
    print("\n=== Fix Summary ===")
    print("BEFORE (Broken):")
    print("  Client called: /v1/ping/ (public endpoint, no auth)")
    print("  Expected: user_info in response")
    print("  Result: 404 Not Found or missing user_info")

    print("\nAFTER (Fixed):")
    print("  Client calls: /v1/users/current (authenticated endpoint)")
    print("  Returns: actual user info with authentication")
    print("  Result: ✅ API key validation works correctly")


if __name__ == "__main__":
    print("Starting API key validation fix test...\n")

    # Test the fix
    success = test_managed_service_fix()

    # Show comparison
    test_comparison()

    if success:
        print("\n✅ SUCCESS: Your inmemory package is working correctly!")
        print("You can now use both:")
        print("- Self-hosted: Memory() - works offline")
        print("- Managed: Inmry() - works with API server")
        sys.exit(0)
    else:
        print("\n❌ Test failed. Check API server and key.")

Starting API key validation fix test...

=== Testing Fixed Managed Service ===
Testing API key validation fix...

1. Testing Inmry initialization with API key...
   API Key: im_c305c0a3aed7...
   Host: http://localhost:8081
✅ Inmry client initialized successfully!
   User ID: d33e5659d6fb47d21d50fc5a163b82cf
   User Info: {'user_id': '68a067ec9f591a3e615a0c1c', 'created_at': None, 'last_used': None, 'usage_count': 0}

2. Testing health check...
✅ Health check successful: {'status': 'healthy', 'service': 'InMemory Core API', 'version': '1.0.0'}

3. Testing memory addition...
✅ Memory added: {'success': True, 'message': 'Memory added successfully', 'memory_id': '3efb629a-7120-4089-9df1-60e593e864b8', 'result': 'I just discovered something new about you!\nTags: test, bug-fix, validation', 'action': 'added', 'metadata': {'tags': ['test', 'bug-fix', 'validation']}}

4. Testing memory search...
✅ Search completed: 0 results

🎉 All tests passed! The API key validation fix is working correctly

SystemExit: 0